In [1]:
import altair as alt
import numpy as np
import oscutils
import scipy
import statsmodels.stats.multitest

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [2]:
inputs = {}

for source in ["mm", "pd"]:
    df = oscutils.\
    load_protein_table(source, "quant", clean=True).\
    drop(columns="sample").\
    set_index(["sample_type", "sample_condition", "sample_num"])
    
    inputs[source] = df

## t-test for differential expression

In [ ]:
def diff_expr_ttest(inputs_dict):
    
    results = {}
    
    for source in inputs_dict.keys():
        df = inputs_dict[source]

        sample_types = []
        prots = []
        pvals = []
        fcs = []

        for sample_type in df.index.get_level_values("sample_type").unique():
            healthy = df.xs((sample_type, "healthy"), level=("sample_type", "sample_condition"), drop_level=True)
            unhealthy = df.xs((sample_type, "unhealthy"), level=("sample_type", "sample_condition"), drop_level=True)

            for prot in healthy.columns:
                a = healthy[prot].dropna()
                b = unhealthy[prot].dropna()

                if sample_type in ["boost", "pbulk"]:
                    min_count = 2
                else:
                    min_coun = 15
                    
                if len(a) >= min_count and len(b) >= min_count:
                    t, p = scipy.stats.mannwhitneyu(x=a, y=b)#, equal_var=True)
                else:
                    p = np.nan
                    
                fc = np.log2(b.mean() / a.mean())

                sample_types.append(sample_type)
                prots.append(prot)
                pvals.append(p)
                fcs.append(fc)

        raw_pvals = pd.DataFrame({
            "sample_type": sample_types,
            "protein": prots,
            "p_uncorrected": pvals,
            "log2_fold_change": fcs,
        })

        pvals = raw_pvals[raw_pvals["p_uncorrected"].notna()]

        # Correct the p values
        reject, pvals_corrected, alphacSidak, alphacBonf = statsmodels.stats.multitest.multipletests(
            pvals=pvals["p_uncorrected"].dropna(),
            alpha=0.05,
            method="fdr_bh",
        )

        pvals = pvals.assign(p_corrected=pvals_corrected) 
        
        results[source] = {
            "pvals": pvals,
            "raw_pvals": raw_pvals,
        }
    
    return results

results = diff_expr_ttest(inputs)

In [ ]:
alpha = 0.05

for source in sources:
    print(source)
    print(results[source]["raw_pvals"].shape)
    print((results[source]["pvals"]["p_uncorrected"] <= alpha).sum())
    print((results[source]["pvals"]["p_corrected"] <= alpha).sum())
    print(results[source]["pvals"]["p_corrected"].min())
    print(results[source]["pvals"][results[source]["pvals"]["p_corrected"] <= alpha])
    print()